In [1]:
print("coucou")

coucou

In [7]:
using Polyhedra
using JuMP
using LinearAlgebra
using Gurobi #, Cbc, GLPK
using Plots
using LaTeXStrings
using JLD

In [8]:
include("../lib/linear_system_v3.jl")
print("Done")

Done

In [9]:
include("../lib/double_integrator_drones.jl")
print("Done")

Done

In [10]:
include("../lib/utils_MILP.jl");
print("Done")

Done

In [11]:
# data of the problem
T=20
N_m=3
N_c=4

enforceRegular=false
minimizeScaling=false #enforceRegular

false

In [12]:
system,polZ0=get_drones(1,T);

if !check(system)
    print("ERROR in dimensions of the system\n")
end

n_x=x_dim(system)
n_y=y_dim(system)
n_z=z_dim(system)
n_u=u_dim(system)

2

In [7]:
# instantiate the ASAP problem as a JuMP model
time_beginning=time_ns()

model,C_bar,S_bar, P_z_all,z_tilde, P_u_all, u_tilde, polEta=instantiateASAP(system,N_m,N_c,T; minimizeScaling=minimizeScaling,enforceRegular=enforceRegular);

time_ending=time_ns()
instantiation_time=(time_ending-time_beginning)/1e9
print("\n\n==== Instantiation time = $(instantiation_time) seconds ====\n\n")



==== Instantiation time = 40.572002431 seconds ====



In [8]:
# Set an optimozer and solve the problem
set_optimizer(model, Gurobi.Optimizer)

time_beginning=time_ns()

optimize!(model)

time_ending=time_ns()
optimization_time=(time_ending-time_beginning)/1e9
print("\n\n==== Optimization time = $(optimization_time) seconds ====\n\n")

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 68414 rows, 42352 columns and 844676 nonzeros
Model fingerprint: 0x2da61585
Variable types: 42312 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+06]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 2e+01]
Presolve removed 55740 rows and 23592 columns
Presolve time: 1.22s
Presolved: 12674 rows, 18760 columns, 137680 nonzeros
Variable types: 18720 continuous, 40 integer (40 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   28315    0.0000000e+00   3.830894e-01   7.565832e+07      5s
Concurrent spin time: 0.51s

Solved with dual simplex

Root relaxation: objective 0.000000e+00, 7528 iterations, 4.11 seconds
Total elapsed time = 17.04s
Total elap

In [9]:
# Extract the solution (gains, sigmas, ...) from the optimal solution of the MILP
F, f, _, sigma_meas_opt, sigma_control_opt=getResultsASAP(model,C_bar,S_bar,N_m,N_c,T;minimizeScaling=minimizeScaling,enforceRegular=enforceRegular);

In [10]:
# compute bounds on z and u and bounds of S_t
lower_bounds_z, upper_bounds_z, lower_bounds_u, upper_bounds_u, lower_bounds_S, upper_bounds_S, lower_bounds_U, upper_bounds_U = computeBounds(system, value.(P_z_all), value.(z_tilde), value.(P_u_all), value.(u_tilde), polEta);

Academic license - for non-commercial use only


 === Compute bounds on z for entry 1 ===

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 248 rows, 124 columns and 248 nonzeros
Model fingerprint: 0xf51b9689
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
Presolve removed 248 rows and 124 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective -1.000000000e+00

User-callback calls 24, time in user-callback 0.00 sec
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 248 rows, 124 columns and 248 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 248 rows, 124 columns and 248 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -6.3290323e+30   1.000000e+31   6.329032e+00      0s
      10   -1.2872820e+00   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.00 seconds
Optimal objective -1.287282011e+00

User-callback calls 518, time in user-callback 0.00 sec
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 248 rows, 124 columns and 248 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4290323e+30   1.200000e+31   7.429032e+00   

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 248 rows, 124 columns and 248 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.5009677e+31   2.000000e+31   1.500968e+01      0s
      20   -1.4367439e+00   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.00 seconds
Optimal objective -1.436743912e+00

User-callback calls 1188, time in user-callback 0.00 sec
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 248 rows, 124 columns and 248 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6109677e+31   2.200000e+31   1.610968e+01  

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 248 rows, 124 columns and 248 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.6154839e+31   3.000000e+31   2.615484e+01      0s
      30   -1.0392773e-01   0.000000e+00   0.000000e+00      0s

Solved in 30 iterations and 0.00 seconds
Optimal objective -1.039277298e-01

User-callback calls 2056, time in user-callback 0.00 sec
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 248 rows, 124 columns and 248 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7254839e+31   3.200000e+31   2.725484e+01  

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 248 rows, 124 columns and 248 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-16, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -8.0742520e+30   3.600000e+31   8.074252e+00      0s
      36    4.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 36 iterations and 0.00 seconds
Optimal objective  4.500000000e+00

User-callback calls 3109, time in user-callback 0.00 sec
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 248 rows, 124 columns and 248 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-16, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0246320e+31   4.200000e+31   1.024632e+01  

...

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 248 rows, 124 columns and 248 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.1263158e+32   2.200000e+31   4.126316e+02      0s
      22   -2.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 22 iterations and 0.00 seconds
Optimal objective -2.000000000e+01

User-callback calls 5138, time in user-callback 0.00 sec
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 248 rows, 124 columns and 248 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1263158e+32   2.200000e+31   4.126316e+02  

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 6e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective -1.500000000e+00

User-callback calls 57, time in user-callback 0.00 sec
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 6e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+30   1.000000e+30   1.000000e+00      0s
       1    1.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved 

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 6e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective -1.500000000e+00

User-callback calls 57, time in user-callback 0.00 sec
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 6e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+30   1.000000e+30   1.000000e+00      0s
       1    6.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved 

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 6e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective -1.500000000e+00

User-callback calls 57, time in user-callback 0.00 sec
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 6e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+30   1.000000e+30   1.000000e+00      0s
       1    6.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved 

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 6e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  4.500000000e+00

User-callback calls 57, time in user-callback 0.00 sec
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 4 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 6e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+30   1.000000e+30   1.000000e+00      0s
       1    5.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved 

In [11]:
# generate trajectories
z,u=simulateTrajectories(system,F,f;numberSamples=1000);

Compute V-representation of W... Done.
Compute V-representation of V... Done.
Compute V-representation of X0... Done.


In [12]:
# plot trajectories
plotList=[]
lab=["x","y"]
# plot z without legends
for component=1:n_z
    p=plot(0:T,z[:,component,:]',label=false)    
    plot!(0:T,lower_bounds_z[component,:],fillrange=upper_bounds_z[component,:],fillalpha=.25,alpha=.25,color=:green,linewidth=0,label=false)

    xlabel!(L"t")
    ylabel!(latexstring("\$p^"*lab[component]*"_t\$"))

    plot!(0:T,lower_bounds_S[component,:],linecolor=:red, label=false)
    plot!(0:T,upper_bounds_S[component,:],linecolor=:red, label=false)

    t_meas=findall(sigma_meas_opt.==1).-1
    scatter!(t_meas,ones(length(t_meas))*maximum(upper_bounds_S[component,:]),label=false,marker=:hexagon,color=:blue)

    t_control=findall(sigma_control_opt.==1).-1
    scatter!(t_control,ones(length(t_control))*minimum(lower_bounds_S[component,:]),color=:orange,label=false)
    xticks!(0:T)
    #xlims!((-0.1,T))
    plotList=[plotList; p]
end

In [13]:
# plot control inputs

# no legend
for component=1:n_u-1
    p=plot(0:T-1,lower_bounds_u[component,:],fillrange=upper_bounds_u[component,:],fillalpha=.25,alpha=.25,color=:green,linewidth=0,label=false)
    plot!(0:T-1,u[:,component,:]',label=false)    
    
    xlabel!(L"t")
    ylabel!(latexstring("\$u^"*lab[component]*"_t\$"))
    
    plot!(0:T-1,ones(T)*lower_bounds_U[component],linecolor=:red, label=false)
    plot!(0:T-1,ones(T)*upper_bounds_U[component],linecolor=:red, label=false)

    t_meas=findall(sigma_meas_opt.==1).-1
    scatter!(t_meas,ones(length(t_meas))*upper_bounds_U[component],label=false,marker=:hexagon,color=:blue)

    t_control=findall(sigma_control_opt.==1).-1
    scatter!(t_control,ones(length(t_control))*lower_bounds_U[component],color=:orange,label=false)
    xticks!(0:T)
    
    plotList=[plotList; p]
end

component=n_u

# Last component, with a legend
p=plot(0:T-1,lower_bounds_u[component,:],fillrange=upper_bounds_u[component,:],fillalpha=.3,alpha=.25,color=:green,linewidth=0,label="Possible values")
plot!(0:T-1,u[:,component,:]',label=false)

xlabel!(L"t")
ylabel!(latexstring("\$u^"*lab[component]*"_t\$"))

plot!(0:T-1,ones(T)*lower_bounds_U[component],linecolor=:red, label=L"\partial\mathcal{Z}_t, \partial\mathcal{U}")
plot!(0:T-1,ones(T)*upper_bounds_U[component],linecolor=:red, label=false)

t_meas=findall(sigma_meas_opt.==1).-1
scatter!(t_meas,ones(length(t_meas))*upper_bounds_U[component],label="Measurement times",marker=:hexagon,color=:blue)

t_control=findall(sigma_control_opt.==1).-1
scatter!(t_control,ones(length(t_control))*lower_bounds_U[component],color=:orange,label="Control times")
xticks!(0:T)
plotList=[plotList; p]

4-element Array{Any,1}:
 Plot{Plots.GRBackend() n=1005}
 Plot{Plots.GRBackend() n=1005}
 Plot{Plots.GRBackend() n=1005}
 Plot{Plots.GRBackend() n=1005}

In [14]:
plot(plotList[1],plotList[2],plotList[3],plotList[4],layout=(4,1)) # The dimension of z and u are hardcoded...
plot!(link=:x)
plot!(legend=:bottomright)
#plot!(legend=:false)
if !enforceRegular
    savefig("results/feasibility.pdf")
    #@save "results/workspace_ASAP.jld"
else
    savefig("results/feasibility_reg.pdf")
    #@save "results/workspace_ASAP_reg.jld"
end
plot!()

In [15]:
initialSet=polyhedron(polZ0)
horizontalRectangle=polyhedron(system.list_polS[1]) # t=0
verticalRectangle=polyhedron(system.list_polS[round(Int8,T/2)+1]) # t=T/2
finalRectangle=polyhedron(system.list_polS[T+1]) # t=T

plot(initialSet,alpha=.5,label=L"\mathcal{X}_{0}^{position}")

lab1=latexstring("\$\\mathcal{Z}_{t}, t=0,\\dots, "*string(convert(Int8,T/2-1))*"\$")
plot!(horizontalRectangle,alpha=.5, label=lab1) # plot safety sets

lab2=latexstring("\$\\mathcal{Z}_{t}, t="*string(convert(Int8,T/2))*",\\dots, "*string(convert(Int8,T-1))*"\$")
plot!(verticalRectangle,alpha=.5, label=lab2)

lab3=latexstring("\$\\mathcal{Z}_{"*string(convert(Int8,T))*"}\$")
plot!(finalRectangle,alpha=.5, label=lab3)

plot!(z[:,1,:]',z[:,2,:]', label=false) # plot trajectories
scatter!(z[:,1,1]',z[:,2,1]',label=false) # plot initial point
scatter!(z[:,1,end]',z[:,2,end]',label=false) # plot the final point

xlabel!(L"x")
ylabel!(L"y")

plot!(legend=:topleft)

if !enforceRegular
    savefig("results/drone.pdf")
else
    savefig("results/drone_reg.pdf")
end

plot!()

InterruptException: InterruptException:

In [ ]:
if !enforceRegular
    @save "results/workspace_ASAP.jld"
else
    @save "results/workspace_ASAP_reg.jld"
end

In [ ]:
using Combinatorics

In [ ]:
numberSchedulings=length(combinations(0:T-1,N_m))*length(combinations(0:T-1,N_c))